In [13]:
import pymysql
from numpy.random import choice
from collections import defaultdict



def mapCategoriesByGameId(categories):
    catMap = defaultdict(list)
    for category in categories:
        catMap[category['gameId']].append(category['id'])
    return catMap

conn = pymysql.connect(host='127.0.0.1', unix_socket='/tmp/mysql.sock', user='root', passwd='root', db='mysql', charset='utf8')
cur = conn.cursor(pymysql.cursors.DictCursor)
cur.execute("USE jeopardy")

cur.execute('SELECT * FROM questions WHERE round = 2')
questions = cur.fetchall()

cur.execute('SELECT * FROM categories WHERE round = 1')
roundOneCategories = mapCategoriesByGameId(cur.fetchall())

cur.execute('SELECT * FROM categories WHERE round = 2')
roundTwoCategories = mapCategoriesByGameId(cur.fetchall())



In [14]:
print(roundOneCategories['3062'])

[987, 988, 989, 990, 991, 992]


In [25]:
print(roundOneCategories['5636'])
print(roundOneCategories['5636'].index(2))

[1, 2, 3, 4, 5, 6]
1


In [33]:
for question in questions[10:]:
    roundOneIndex = roundOneCategories[str(question['gameId'])].index(int(question['categoryId']))
    correctCategoryId = roundTwoCategories[str(question['gameId'])][roundOneIndex]
    cur.execute('UPDATE questions SET categoryId = %s WHERE id = %s', (correctCategoryId, question['id']))
    conn.commit()